<a href="https://colab.research.google.com/github/ARNAB5671/cs253_python_assignment/blob/main/cs253_A3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'who-is-the-real-winner:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F72632%2F8059709%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240421%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240421T212956Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D2772c06e4a696239b37a17ec896bee2dd942c3d68d750be9d91dbd2816a5a305c33224596ebdd3bcf68f3b1f1325c4735c2fdf2ee9303d39f3964e0de6bed533c7a2a91c716504af7ff21c884889d95c9b9bc2f6b14f53e72fd052ee15de057696c811914cf689a842b8d54c43f512afd59bda3e9ff2b33aceb450e20b8a3c177a6f4268457d647884bb73a4e763e951b474cbf94eef5b4c68cb2cc631b66062f612a09131e08a68004422531edd6ad282aadff379f6dc1e8bc79ee0beb0f23ece90a33a34f65c97138104b60f3e18534740b1c41a2af8ab20d14eb17ebb520caadb5db3915bb7cfd3ddfc9fd94d1f5263e608b493faeff71439b84635e27021'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df1 = pd.read_csv('/kaggle/input/who-is-the-real-winner/train.csv')
df2 = pd.read_csv('/kaggle/input/who-is-the-real-winner/test.csv')
submission = pd.DataFrame(columns=['ID','Education'])
submission['ID']  = df2['ID']
df1

In [ ]:
features=['Party','Criminal Case','state']
X = df1[features]
y = df1.Education
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)
for i in X.columns:
    if X[i].dtype=='object':
        X[i] = label_encoder.fit_transform(X[i])

In [ ]:
from sklearn.ensemble import RandomForestClassifier
newmodel = RandomForestClassifier(random_state=50)
newmodel.fit(X,y)

In [ ]:
df2

In [ ]:
df2.drop('ID',axis=1,inplace=True)
df2.drop('Candidate',axis=1,inplace=True)
df2.drop('Constituency ∇',axis=1,inplace=True)
df2.drop('Liabilities',axis=1,inplace=True)
df2.drop('Total Assets',axis=1,inplace=True)



In [ ]:
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)
for i in df2.columns:
    if df2[i].dtype=='object':
        df2[i] = label_encoder.fit_transform(df2[i])
df2

In [ ]:
predictions = newmodel.predict(df2)
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df1.Education)
submission['Education'] = label_encoder.inverse_transform(predictions)
submission

In [ ]:
submission.to_csv('/kaggle/working/submission.csv',index=False)